In [1]:
train_path=r"C:\Users\amc\Downloads\archive (15)\Celebrity Faces Dataset\train_set"

In [3]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [ ]:
# ✅ Data Augmentation + Validation Split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

# ✅ Class Names
class_names = list(train_set.class_indices.keys())
print("Classes:", class_names)

# ✅ Pure CNN Architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(len(class_names), activation='softmax')  # Multi-class output
])

# ✅ Compile Model
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ✅ Callbacks to Avoid Overfitting
callbacks = [
    EarlyStopping(patience=7, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.3, patience=3, verbose=1)
]

# ✅ Train Model
history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set,
    callbacks=callbacks,
    verbose=1
)

# ✅ Final Evaluation
loss, accuracy = model.evaluate(val_set, verbose=0)
print(f"\n✅ Validation Accuracy: {accuracy * 100:.2f}%")

# ✅ Random Prediction
img_batch, label_batch = next(val_set)
random_index = random.randint(0, len(img_batch)-1)
img = np.expand_dims(img_batch[random_index], axis=0)
pred = model.predict(img)

true_class = class_names[np.argmax(label_batch[random_index])]
pred_class = class_names[np.argmax(pred)]

print("\n🔍 Random Prediction")
print("Actual Actor:", true_class)
print("Predicted Actor:", pred_class)

Found 480 images belonging to 5 classes.
Found 120 images belonging to 5 classes.
Classes: ['Angelina Jolie', 'Brad Pitt', 'Leonardo DiCaprio', 'Scarlett Johansson', 'Tom Cruise']


E:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 148, 148, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 72, 72, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 34, 34, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       9,470,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,598,917 (36.62 MB)

 Trainable params: 9,597,957 (36.61 MB)

 Non-trainable params: 960 (3.75 KB)

E:\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.2750 - loss: 2.1152 - val_accuracy: 0.3333 - val_loss: 1.7061 - learning_rate: 5.0000e-04
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4021 - loss: 1.6390 - val_accuracy: 0.1917 - val_loss: 1.8154 - learning_rate: 5.0000e-04
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4396 - loss: 1.5354 - val_accuracy: 0.1917 - val_loss: 1.9222 - learning_rate: 5.0000e-04
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4671 - loss: 1.4986
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
15/15 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.4750 - loss: 1.5117 - val_accuracy: 0.1667 - val_loss: 2.1265 - learning_rate: 5.0000e-04
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4625 - loss: 1.4404 - val_accuracy: 0.1917 - val_loss: 2.3508 - learning_rate: 1.5000e-04
Epoch 6/30
 8/15 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.4569 - loss: 1.499